### Building a Random Forest Classifier with Spark in Batch Mode

It's time to get into Apache Spark, and the best way to do so is by experimenting with some code.

We'll start by going over some terms, explain why we chose the dataset we'll use, and finally, build a classifier to identify movement type.

#### How will we use Spark in this example?

For this example, we'll use a local, standalone Apache Spark. In fact, we're going to use Spark in this way through most of the examples and exercises. By starting with a simple deployment, we can focus on the details of developing machine learning solutions with Spark and postpone the complexities of deploying a cluster until after we understand the details of implementing machine learning solutions with Spark.

We'll begin our machine learning with Spark exercise by using it in __batch mode__. This mode is most similar to the approach you've taken so far in this course - load a dataset, clean it, then run models and evaluate their performance.

__Streaming mode__ differs from batch mode in that, under streaming mode, you do not have access to all of the data. Instead, you ingest data into Spark over a period of time. As each data element arrives, you perform data science tasks (clean, classify, etc) on the new data, then archive the result. Don't worry about the details of this now - we'll cover it soon enough. For now, you only need to be aware that there are two modes we can work from in Spark.

Once you're confident you can use local, standalone Spark in both batch and streaming mode, we'll work through how to create, deploy, and manage a Spark cluster in the cloud.

#### Our Imports:

Spark works a little different than the data science stack you've worked with already. In the data science stack, you are writing Python code that both executes immediately and also executes locally on the device you're using.

Spark, on the other hand, lives outside the Python environment. We treat it as a server that we pass instructions to; the Spark server performs the calculations and then reports back to our notebook. Additionally, Spark uses the concept of a __pipeline__ in which we configure a number of steps, that are only executed when we tell the server to run.

There are a number of benefits to this approach for big data. First of all, by using the server paradigm, we can run locally with a smaller set of data when we're building and testing code. Then, when we've successfully set up our model, deploying to production means we point our Python code to a remote server that is configured to handle larger datasets. Second, the pipeline approach allows us to configure models and run them through multiple steps more efficiently on big data.

In [1]:
# these imports allow us to set up our Python connection to the Spark server.
# it also allows us to load a dataframe.
from pyspark import SparkContext
from pyspark.sql import SparkSession

# these imports are how we build and manager our data science processes: cleaning data, preparing a model,
# executing the model, and evaluating the model.
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer, VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

from pyspark.sql.functions import isnan, when, count, col

It's always a good idea to review the documentation for modules, methods, and functions we'll use. Everything we look at here can be found in the Apache Spark documentation.

In particular, you should familiarize yourself with the:
* [Spark Overview](https://spark.apache.org/docs/2.1.0/index.html)
* [Python API Documentation](https://spark.apache.org/docs/2.1.0/api/python/index.html)

For our work here, it's also good to read about random forests in Spark [here](https://spark.apache.org/docs/2.1.0/ml-classification-regression.html#random-forests).

In [2]:
# we use a set of constants for clarity and simplicity in managing the notebook.
# this allows you to refer back to this cell at any time if you need to either confirm or modify any of these values.

CSV_PATH = "/home/ds/notebooks/datasets/UCI_HAR/allData.csv"
CSV_ACTIVITY_LABEL_PATH = "/home/ds/notebooks/datasets/UCI_HAR/activity_labels.csv"
APP_NAME = "UCI HAR Random Forest Example"
SPARK_URL = "local[*]"
RANDOM_SEED = 141107
TRAINING_DATA_RATIO = 0.8
RF_NUM_TREES = 10
RF_MAX_DEPTH = 4
RF_NUM_BINS = 32

#### Getting Started:

1. Create the connection to the Spark server using the SparkSession.
2. After that's done, import two datasets and do some cleaning and validation before we configure our model.

We import two datasets:
    1. `activity_labels`: a mapping of the classifier labels used in the datasets.
    2. `df`: a generic name for the dataset we're using.

#### ASIDE: THE UCI HAR Dataset

We'll be using a slightly modified version of the UCI HAR dataset. This is a particularly nice dataset to use when learning Spark since everything is already numeric, and there is minimal cleaning needed.

This dataset contains data from experiments on 30 subjects who wore a smartphone attached to their waist. They performed six activities:
* WALKING
* WALKING UP STAIRS
* WALKING DOWN STAIRS
* SITTING
* STANDING
* LAYING

The activities were recorded with video, which enabled time-stamped correlation with motion values collected from the smartphones. In particular, 3-axial linear acceleration and 3-axial angular velocity were captured from the device's accelerometer and gyroscope. This resulted in a 561-value feature vector, plus its corresponding activity label.

The source dataset is available [here](https://archive.ics.uci.edu/ml/datasets/human+activity+recognition+using+smartphones). 

If you download this, you'll see that there are actually four datasets:
1. __Train__
    * X_features
    * Y_labels
2. __Test__
    * X_features
    * Y_labels
    
To use this in Spark, we do a little bit of preparation outside this example. We'll describe the steps, in the event you choose to try to prep the data yourself (it's always a good exercise to flex your data munging muscles when you can).

Here's what we did to generate the `allData.csv` file:
1. The source files as provided are space-delimited. And, unfortunately, the files are inconsistent with the spacing. Most of the time there are only single space delimiters, but you find double spaces intermittently too. Those double spaces add extraneous columns to the dataset and create problems for our classifier.
In order to get around this, we replaced the spaces with commas. It's not necessary but mainly for personal preferences. We also do a string replace to get rid of all double-delimiters and replace with single delimiters. 
When you've completed this step, you can import and should see that you have __561__ unique feature columns in both the train and test datasets.

2. We also merge the features and labels datasets, then append the test dataset onto the train dataset.

The final dataset should have __10,299__ rows and __562__ columns. Everything should be numeric - the labels will be integers and the features will be doubles. Because the source data is already numeric, it's simpler to build and demonstrate our classifier in Spark.

#### Why didn't we use the Lending Club dataset from our earlier random forest exercise?

Great question! The simple answer is that we could have used that dataset for this exercise. However, Spark explicitly demands that all inputs to the random forest classifier be indexed labels and indexed numeric feature vectors. The Lending Club data doesn't easily translate from the delivered format into Spark-ready formats. 

Thus, rather than spend 90% of an exercise on data cleaning and preparation, we'll use the UCI HAR dataset. It requires only a minimal amount of preparation and cleaning, so we can spend most of our time on the relevant Spark material.

In [3]:
spark = SparkSession.builder.appName(APP_NAME).master(SPARK_URL).getOrCreate()

activity_labels = spark.read.options(inferschema = "true").csv(CSV_ACTIVITY_LABEL_PATH)

df = spark.read.options(inferschema = "true").csv(CSV_PATH)

#### Data Validation

As mentioned above, if we cleaned and prepared the data properly, the dataset will meet the following three conditions.

Look over the datasets for:
1. Final shape should be 10299 rows by 562 columns
2. All feature columns should be doubles
3. There should be no nulls
    * This is important since Spark will fail to build our vector variables that we need in our classifier.

In [4]:
# As we get going, we first create some diagnostic variables to conduct our tests.

# Testing for data types
# Use a list comprehension to grab the column names that all have the data type 'double'
double_cols = [col[0] for col in df.dtypes if col[1] == 'double']

# Testing for nulls in columns. 
# We use the dataframe select method to build a list that is then converted to a Python dict.
# This way it's easy to sum up the nulls in a moment when we're actually testing for presence of nulls. 
null_counts = df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) 
                         for c in df.columns]).toPandas().to_dict(orient='records')

For our example, since we're using a notebook, we'll just pretty things up by using `print` statements to confirm the tests we mentioned above.

In [5]:
print(f"Dataset shape is {df.count():d} rows by {len(df.columns):d} columns.")

Dataset shape is 10299 rows by 562 columns.


In [6]:
print(f"{len(double_cols):d} columns out of {len(df.columns):d} total are type double.")

561 columns out of 562 total are type double.


In [7]:
print(f"There are {sum(null_counts[0].values()):d} null values in the dataset.")

There are 0 null values in the dataset.


#### Setting up and running a classifier in Spark

Assuming that the data is now clean, we are now ready to reshape the data and run the random forest model.

In Spark, we manipulate the data to work in a Spark pipeline, define each of the steps in the pipeline, chain them together in a pipeline, and finally run the pipeline.

The Apache Spark classifiers expect two columns of input:
1. __labels__: an indexed set of numeric variables that represent the classification from the set of features we provide.
2. __features__: An indexed, vector variable that contains all of the feature values in each row. 

In order to do this, we need to create these two columns from our dataset - the data is there, but not yet in a format we can use in the classifier.

To create the indexed labels column, we create a column called `indexedLabel` using the `StringIndexer` method.
1. We use the column `_c0` as the source for our label index since that contains our labels. The column contains only one value per index.
    
To create the indexed features column, we need to do two steps:
1. Create the vector of features using the `VectorAssembler` method.
    * We need to use all numeric columns in our data frame - there are 561 - to create this vector.
    * The vector assembler creates a new column called `features`; each row of this column contains a 561-element vector that is built from the 561 features in the dataset.

2. Complete the data preparation by creating an indexed vector from the `features` column.
    * This vector is called `indexedFeatures`.
    
Since the classifier expects indexed labels and an indexed vector column of data, we use the `indexedLabel` and `indexedFeatures` as inputs to our random forest classifier.

In [8]:
# Generate our feature vector.
# Note that we're doing the work on the `df` object - we don't create new dataframes, 
# just add columns to the one we already are using.

# the transform method performs the act of creating the column.

df = VectorAssembler(inputCols=double_cols, outputCol="features").transform(df)

Let's confirm that the features are there.

It's easy to do this in Apache Spark using the `select` and `show` methods on the dataframe.  

In [9]:
df.select("_c0", "features").show(5)

+---+--------------------+
|_c0|            features|
+---+--------------------+
|  5|[0.289,-0.0203,-0...|
|  5|[0.278,-0.0164,-0...|
|  5|[0.28,-0.0195,-0....|
|  5|[0.279,-0.0262,-0...|
|  5|[0.277,-0.0166,-0...|
+---+--------------------+
only showing top 5 rows



Now we want to build the indexers, split our data for training and testing, define our model, and finally chain everything together into a pipeline.

* __It is important to note - when we execute this cell, we're not actually running our model. We're only defining its parameters in this cell.__

In [10]:
# Build the training indexers / split data / classifier
# first we'll generate a labelIndexer
labelIndexer = StringIndexer(inputCol="_c0", outputCol="indexedLabel").fit(df)

# now generate the indexed feature vector.
featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(df)
    
# Split the data into training and validation sets (30% held out for testing)
(trainingData, testData) = df.randomSplit([TRAINING_DATA_RATIO, 1 - TRAINING_DATA_RATIO])

# Train a RandomForest model.
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", numTrees=RF_NUM_TREES)

# Chain indexers and forest in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, rf])

This next cell runs the pipeline - delivering a trained model at the end of the process.

In [11]:
# Train model.  This also runs the indexers.
model = pipeline.fit(trainingData)

It is now easy to test our model and make predictions, simply by using the model's `transform` method on the `testData` dataset.

In [12]:
# Make predictions.
predictions = model.transform(testData)

#### Evaluate the model

We can use the MulticlassClassificationEvaluator to test the model's accuracy.

In [13]:
# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

print(f"Test Error = {(1.0 - accuracy):g}")
print(f"Accuracy = {accuracy:g}")

Test Error = 0.0811202
Accuracy = 0.91888


#### Next Steps:

So we've seen how to set up some data and build a classifier in Spark. You might want to play around with this notebook and learn more about how Spark works.

Some ideas:
1. Look at the set of labels, and see if there are features that might be better if combined. Spark has a means to map values into a new column.

2. Identify the most important features among the 561 source features (using PCA or something similar); reduce the feature set and see if the model performs better.

3. Modify the settings of the random forest to see if the performance improves.

4. Use Spark's tools to find other techniques to evaluate the performance of your model - see if you can figure out how to generate a ROC plot, find the AUC value, or plot a confusion matrix.